In [1]:
from selenium.webdriver import ChromeOptions

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

import time

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

# = = =  = = =  = = =

import pandas as pd

In [2]:
option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_argument('--no-sandbox')
# option.add_argument('--headless=new')
# option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument(get_header(method='selenium'))
option.add_argument(get_proxy(method='selenium'))
option.page_load_strategy = 'normal'

driver = Chrome(service=Service('../chromedriver'), options=option)
driver.maximize_window()

# = = =  = = =  = = =

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        driver.get('https://www.amazon.com/?language=en_US')
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0.2 * document.body.scrollHeight);")
        time.sleep(1.5)
        driver.execute_script("window.scrollTo(0, 0.45 * document.body.scrollHeight);")
        time.sleep(0.7)
        driver.execute_script("window.scrollTo(0, 0.7 * document.body.scrollHeight);")
        time.sleep(1.2)
        driver.execute_script("window.scrollTo(0, 0.95 * document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0);")

        break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

driver

1


<selenium.webdriver.chrome.webdriver.WebDriver (session="273babdeae40ab0bf38b8ddec24be2bc")>

In [3]:
list_url = []

list_url

[]

# 1. BSR

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

import json

soup = BeautifulSoup(driver.page_source, 'lxml')
html = etree.HTML(str(soup))

list_dict = json.loads(html.xpath('//div[@data-client-recs-list]/@data-client-recs-list')[0])
list_url_temp = [f'''https://www.amazon.com/dp/{dict_['id']}''' for dict_ in list_dict]

len(list_url_temp), list_url_temp[-1]

(50, 'https://www.amazon.com/dp/B0CNJZ51QG')

In [7]:
list_url.extend(list_url_temp)

len(list_url)

100

# 2. temp

In [ ]:
from bs4 import BeautifulSoup
from lxml import etree

import json

soup = BeautifulSoup(driver.page_source, 'lxml')
html = etree.HTML(str(soup))

list_url_temp = [f'https://www.amazon.com{href.strip()}' for href in html.xpath('//li[@data-testid="product-grid-item"]/div/a/@href')]

len(list_url_temp), list_url_temp[-1]

In [ ]:
list_url.extend(list_url_temp)

len(list_url)

# = = =  = = =  = = =

In [8]:
driver.quit()

print('Data Outputting...')
if list_url:
    df = pd.DataFrame({'No': range(1, len(list_url) + 1),
                       'Url': list_url})
    df.to_excel('./file/1.url.xlsx', index=False)
print('Output Finished !')
print()

df

Data Outputting...
Output Finished !



,No,Url
0,1,https://www.amazon.com/dp/B08W98H7H7
1,2,https://www.amazon.com/dp/B093S8XVSL
2,3,https://www.amazon.com/dp/B08W9MX59Y
3,4,https://www.amazon.com/dp/B08TTC1HQ3
4,5,https://www.amazon.com/dp/B0BPCP4XQN
...,...,...
95,96,https://www.amazon.com/dp/B0BT22K3G7
96,97,https://www.amazon.com/dp/B0BH4ZXNKP
97,98,https://www.amazon.com/dp/B0BY213GX3
98,99,https://www.amazon.com/dp/B095C2ZSK1
